In [2]:
import requests
from bs4 import BeautifulSoup
import math
import concurrent.futures
import json
import os
import time
import pandas as pd

In [3]:
with open("all_10digit_info.csv", "r", encoding="utf-8") as f:
    links = [line.strip() for line in f if line.strip()]
print(f"Total links for craw: {len(links)-1}")

Total links for craw: 11982


In [4]:
df = pd.read_csv("all_10digit_info.csv" , encoding="utf-8",dtype=str)

In [5]:
links = df['link'].tolist()

In [6]:
output_dir = "org2"
os.makedirs(output_dir, exist_ok=True)

state_file_template = os.path.join(output_dir, "state_{}.json")
output_file_template = os.path.join(output_dir, "output_{}.txt")
error_file_template = os.path.join(output_dir, "error_{}.txt")

def save_state(thread_index, start_index):
    """Lưu trạng thái hiện tại của thread vào file."""
    with open(state_file_template.format(thread_index), "w") as f:
        json.dump({"start_index": start_index}, f)

def load_state(thread_index):
    """Tải trạng thái đã lưu từ file."""
    try:
        with open(state_file_template.format(thread_index), "r") as f:
            states = json.load(f)
            return states.get("start_index", 0)
    except (FileNotFoundError, json.JSONDecodeError):
        return 0  # Nếu chưa có file hoặc lỗi load state, bắt đầu từ đầu

def fetch_with_retry(url, retries=5, delay=5):
    """Gửi request với retry khi lỗi."""
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=60)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                return response
        except requests.RequestException as e:
            print(f"Request error: {e} - Retrying {attempt+1}/{retries}...")
            time.sleep(delay)
    return None

In [22]:
def scrape_links(sub_links, thread_index, start_index):
    """Xử lý từng nhóm link và lưu kết quả."""
    batch_results = []

    for idx, link in enumerate(sub_links[start_index:], start=start_index):
        hs10digit = link[-10:]
        print(f"Thread {thread_index} - Processing: {hs10digit}")

        response = fetch_with_retry(link)
        if response is None:
            error_msg = {"link": link, "error": "Failed to fetch after 3 retries"}
            print(f"Thread {thread_index} - {error_msg}") 
            with open(error_file_template.format(thread_index), "a", encoding='utf-8') as f:
                f.write(json.dumps(error_msg, ensure_ascii=False) + "\n")
            continue

        soup = BeautifulSoup(response.text, 'html.parser')
        tables = soup.find_all('table', class_="zebra")

        if len(tables) < 2:
            print(f"Thread {thread_index} - No valid tables found for {hs10digit}")
            continue

        table13_trs, table8_trs = tables[0].find_all('tr'), tables[1].find_all('tr')

        if len(table13_trs) > 1:
            for tr in table13_trs[1:]:
                hs13info ={
                        "hs10digit": hs10digit,
                        "hs13digit": tr.find_all('td')[0].text,
                        "hs13_commodity_name": tr.find_all('td')[1].get_text().strip()
                    }
                batch_results.append(json.dumps(hs13info, ensure_ascii=False))
        if len(table8_trs) > 1:
            for tr in table8_trs[1:]:
                hs8info = {
                        "hs10digit": hs10digit,
                        "hs8digit": tr.find_all('td')[0].text,
                        "hs8_commodity_name": tr.find_all('td')[1].get_text().strip()
                    }
                batch_results.append(json.dumps(hs8info, ensure_ascii=False))

        if len(batch_results) > 0:
            with open(output_file_template.format(thread_index), "a", encoding='utf-8') as f:
                f.write("\n".join(batch_results) + "\n")
            batch_results.clear()

        save_state(thread_index, idx + 1)

    # Ghi dữ liệu còn lại trong batch
    if batch_results:
        with open(output_file_template.format(thread_index), "a", encoding='utf-8') as f:
            f.writelines("\n".join(batch_results) + "\n")


In [7]:
chunk_size = 1000
link_chunks = [links[i:i + chunk_size] for i in range(0, len(links), chunk_size)]

In [ ]:
# Tải trạng thái đã lưu
saved_states = [load_state(i) for i in range(len(link_chunks))]

# Threading
with concurrent.futures.ThreadPoolExecutor(max_workers=min(len(link_chunks), 15)) as executor:
    futures = {
        executor.submit(scrape_links, chunk, idx, saved_states[idx]): idx
        for idx, chunk in enumerate(link_chunks)
    }
    for future in concurrent.futures.as_completed(futures):
        thread_index = futures[future]
        try:
            future.result()
        except Exception as e:
            print(f"Thread {thread_index} encountered an error: {e}")

Thread 11 - Processing: 9801001000
Thread 11 - Processing: 9801009000
Thread 11 - Processing: 9801301000
Thread 11 - Processing: 9801309000
Thread 11 - Processing: 9803001000
Thread 11 - Processing: 9803002000
Thread 11 - Processing: 9803003000
Thread 11 - Processing: 9803009000


In [24]:
# link  = 'https://www.transcustoms.com/China_HS_Code/China_HS_code.asp?HS_Code=0207142902'
# hs10digit = link[-10:]
# # print(f"Thread {thread_index} - Processing: {hs4digit}")

# response = fetch_with_retry(link)
# # if response is None:
# #     error_list.append({"link": link, "error": "Failed to fetch"})
# #     continue
# soup = BeautifulSoup(response.text, 'html.parser')
# tables = soup.find_all('table', class_="zebra")
# table13 = tables[0]
# table8 = tables[1]
# # if not info:
# #     continue
# table13_trs = table13.find_all('tr') 
# table8_trs = table8.find_all('tr') 
# hs13info = []
# if len(table13_trs) == 1:
#     True
# else:
#     for tr in table13_trs[1:]:
#         hs13row = {
#             "hs10digit": hs10digit,
#             "hs13digit": tr.find_all('td')[0].text,
#             "hs13_commodity_name": tr.find_all('td')[1].get_text().replace('\n','').replace('\r','').replace('\t','').strip()
#         }
#         hs13info.append(hs13row)
# hs8info = []
# if len(table8_trs) == 1:
#     True
# else:
#     for tr in table8_trs[1:]:
#         hs8row = {
#             "hs10digit": hs10digit,
#             "hs8digit": tr.find_all('td')[0].text,
#             "hs8_commodity_name": tr.find_all('td')[1].get_text().replace('\n','').replace('\r','').replace('\t','').strip()
#         }
#         hs8info.append(hs8row)

In [ ]:
import pandas as pd
directory = 'org2/'  

err_file_names = [f'error_stt_{i}.txt' for i in range(len(link_chunks))]
all_err_data = []
try:
    for file_name in err_file_names:
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = [json.loads(line) for line in file]
            all_err_data.extend(data) 
            
    df = pd.DataFrame(all_err_data)
    print(len(df))

    df.to_csv('all_craw_13_8_digit_error_stt.csv', index=False, encoding='utf-8')
except Exception as e:
    print(e)

output_file_names = [f'output_{i}.txt' for i in range(len(link_chunks))]
all_output_data = []
try:
# Đọc và xử lý từng file
    for file_name in output_file_names:
        file_path = os.path.join(directory, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            data = [json.loads(line) for line in file]
            all_output_data.extend(data)  
    df2 = pd.DataFrame(all_output_data)
    print(len(df2))

    df2.to_csv('all_13_8_digit_info.csv', index=False, encoding='utf-8')
except Exception as e:
    print(e)

[Errno 2] No such file or directory: 'org/error_stt_0.txt'
173848


In [9]:
import pandas as pd

# Đọc file CSV với encoding UTF-8
df2 = pd.read_csv("all_13_8_digit_info.csv", dtype=str, encoding="utf-8")

# Ghi file Excel với `openpyxl`
with pd.ExcelWriter("all_13_8_digit_info.xlsx", engine="openpyxl") as writer:
    df2.to_excel(writer, sheet_name="Sheet1", index=False)
    
    # Lấy sheet và workbook
    workbook = writer.book
    worksheet = writer.sheets["Sheet1"]
    
    # Định dạng toàn bộ cột là "Text"
    for col in worksheet.iter_cols():
        for cell in col:
            cell.number_format = "@"

print("Đã lưu file Excel với Unicode và định dạng Text thành công!")


Đã lưu file Excel với Unicode và định dạng Text thành công!
